In [2]:
#Import Modules
import pandas as pd
import csv
import mysql.connector
from getpass import getpass
from mysql.connector import connect, Error

In [3]:
#Mysql Connect
try:
    with connect(
        host = "localhost",
        user = input ("Enter user name:"),
        password = getpass ("Enter Password:"),
        database = "Expense_Tracker"
    )as connection:
        print(connection)

except Error as e:
    print(e)

In [4]:
connection.reconnect()

In [4]:
df = pd.read_csv ("Per_Exp_Track.csv")
print(df.head())
print(f"Total rows in CSV: {len(df)}")

         Date   Categories  Paymentmode Description   Amount  Cashback
0  2024-01-10  Subcription          UPI     Spotify   546.18      6.38
1  2024-01-10         Fees  Credit Card     Termfee   958.56      5.59
2  2024-01-07         Fees         Cash      Semfee  1571.84      0.55
3  2024-01-19  Subcription  Credit Card     Spotify   239.68      5.63
4  2024-01-14        Bills         NEFT          EB  1236.30      3.12
Total rows in CSV: 1200


In [63]:
#Create Database
create_db_query = "CREATE DATABASE IF NOT EXISTS Expense_Tracker"

with connection.cursor() as cursor:
    cursor.execute(create_db_query)

In [121]:
# create table

create_tables = """
CREATE TABLE per_exp_track(
    ID INT AUTO_INCREMENT PRIMARY KEY,
    Date DATE NOT NULL,
    Categories VARCHAR(300),
    Paymentmode VARCHAR(100),
    Description TEXT,
    Amount DECIMAL(10,2) NOT NULL,
    Cashback DECIMAL(10, 2) NOT NULL
)
"""
with connection.cursor() as cursor:
    cursor.execute(create_tables, multi=True)
    connection.commit()

ProgrammingError: 1050 (42S01): Table 'per_exp_track' already exists

In [59]:
#Data insert into Mysql table

insert_query = """
INSERT INTO per_exp_track(Date, Categories, Paymentmode, Description, Amount, Cashback)
VALUES(%s, %s, %s, %s, %s, %s)
"""
#Insert Query
batch_size = 1200
data_to_insert = []
for i, row in df.iterrows():
    data_to_insert.append((row['Date'], row['Categories'], row['Paymentmode'], row['Description'], row['Amount'], row['Cashback'],))

with connection.cursor() as cursor:
    cursor.executemany(insert_query, data_to_insert)
    connection.commit()

print (f"{cursor.rowcount} CSV data succesfully inserted Mysql Databases")

1200 CSV data succesfully inserted Mysql Databases


In [33]:
connection.reconnect

<bound method MySQLConnectionAbstract.reconnect of <mysql.connector.connection_cext.CMySQLConnection object at 0x0000020B81CAD790>>

In [12]:
#Delete the Duplicate Entries
delete_dup_datas = """
SELECT Date, Categories, Paymentmode, Description, Amount, Cashback, COUNT(*) AS DuplicateCount
FROM per_exp_track
GROUP BY Date, Categories, Paymentmode, Description, Amount, Cashback
HAVING COUNT(*) > 1
"""

with connection.cursor() as cursor:
    cursor.execute(delete_dup_datas)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(datetime.date(2024, 6, 12), 'Subscription', 'UPI', 'Netflix', Decimal('639.00'), Decimal('5.00'), 2)


In [35]:
#Total Expense for the year
total_exp = """
SELECT SUM(Amount) AS TotalExpense FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(total_exp)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)


(Decimal('1203192.05'),)


In [36]:
#Total Cashback
cb_query = """
SELECT SUM(Cashback) AS TotalCashback FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(cb_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(Decimal('6048.34'),)


In [31]:
#Max Amount Query
max_query = """
SELECT MAX(Amount) AS Maxexpamount FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(max_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(Decimal('1999.62'),)


In [37]:
#Avg Amount Query
avg_query = """
SELECT AVG(Amount) AS Avgamount FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(avg_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(Decimal('1000.159643'),)


In [54]:
#Total Expenses for 12 Months
Total_exp_month = """
SELECT DATE_FORMAT(Date, '%Y-%m') AS Month, SUM(Amount) AS Total_Expense 
FROM per_exp_track
WHERE Date >= DATE_SUB(CURDATE(), INTERVAL 12 MONTH) 
GROUP BY DATE_FORMAT(Date, '%Y-%m') 
ORDER BY Month ASC
"""

with connection.cursor() as cursor:
    cursor.execute(Total_exp_month)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('2024-01', Decimal('62775.94'))
('2024-02', Decimal('97239.92'))
('2024-03', Decimal('106992.85'))
('2024-04', Decimal('98568.74'))
('2024-05', Decimal('109724.92'))
('2024-06', Decimal('107992.33'))
('2024-07', Decimal('91046.20'))
('2024-08', Decimal('99067.16'))
('2024-09', Decimal('97970.11'))
('2024-10', Decimal('99642.56'))
('2024-11', Decimal('104218.67'))
('2024-12', Decimal('91670.97'))


In [38]:
#>Amount Rs.1000/-
great_amount_query = """
SELECT * FROM per_exp_track
WHERE Amount > 1000
"""

with connection.cursor() as cursor:
    cursor.execute(great_amount_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(1, datetime.date(2024, 1, 3), 'Subcription', 'Credit Card', 'Amazonprime', Decimal('1894.91'), Decimal('6.23'))
(4, datetime.date(2024, 1, 12), 'Vegetables', 'UPI', 'Weekly purchase', Decimal('1604.39'), Decimal('8.31'))
(5, datetime.date(2024, 1, 4), 'Health', 'Cash', 'Health Drinks', Decimal('1178.74'), Decimal('9.33'))
(7, datetime.date(2024, 1, 11), 'Bills', 'NEFT', 'EB', Decimal('1732.55'), Decimal('7.67'))
(9, datetime.date(2024, 1, 31), 'Vegetables', 'UPI', 'Weekly purchase', Decimal('1120.84'), Decimal('5.17'))
(10, datetime.date(2024, 1, 30), 'Subcription', 'NEFT', 'Amazonprime', Decimal('1007.73'), Decimal('9.41'))
(11, datetime.date(2024, 1, 9), 'Subcription', 'Cash', 'Amazonprime', Decimal('1550.59'), Decimal('7.93'))
(15, datetime.date(2024, 1, 22), 'Vegetables', 'UPI', 'Weekly purchase', Decimal('1888.09'), Decimal('4.80'))
(17, datetime.date(2024, 1, 10), 'Shopping', 'UPI', 'New Furnitures', Decimal('1120.00'), Decimal('2.81'))
(18, datetime.date(2024, 1, 26), 'Shopping

In [15]:
#Payment UPI
Pay_Cash_query = """
SELECT * FROM per_exp_track
WHERE Paymentmode = "Cash"
"""

with connection.cursor() as cursor:
    cursor.execute(Pay_Cash_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(5, datetime.date(2024, 1, 4), 'Health', 'Cash', 'Health Drinks', Decimal('1178.74'), Decimal('9.33'))
(11, datetime.date(2024, 1, 9), 'Subcription', 'Cash', 'Amazonprime', Decimal('1550.59'), Decimal('7.93'))
(14, datetime.date(2024, 1, 8), 'Vegetables', 'Cash', 'Weekly purchase', Decimal('607.30'), Decimal('7.01'))
(18, datetime.date(2024, 1, 26), 'Shopping', 'Cash', 'New Furnitures', Decimal('1737.12'), Decimal('7.24'))
(19, datetime.date(2024, 1, 5), 'Vegetables', 'Cash', 'Weekly purchase', Decimal('13.46'), Decimal('0.95'))
(20, datetime.date(2024, 1, 28), 'Fees', 'Cash', 'Termfee', Decimal('403.18'), Decimal('2.37'))
(21, datetime.date(2024, 1, 26), 'Groceries', 'Cash', 'Weekly purchase', Decimal('1352.14'), Decimal('1.02'))
(28, datetime.date(2024, 1, 23), 'Bills', 'Cash', 'Cable', Decimal('1315.76'), Decimal('9.79'))
(29, datetime.date(2024, 1, 12), 'Investments', 'Cash', 'Insurance', Decimal('1936.26'), Decimal('7.66'))
(34, datetime.date(2024, 1, 22), 'Investments', 'Cash', '

In [17]:
#Amount Desc Order
Desc_Amt_query = """
SELECT * FROM per_exp_track
ORDER BY Amount DESC 
LIMIT 20
"""

with connection.cursor() as cursor:
    cursor.execute(Desc_Amt_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(258, datetime.date(2024, 3, 26), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('1999.62'), Decimal('8.91'))
(991, datetime.date(2024, 10, 10), 'Fees', 'NEFT', 'Coursefee', Decimal('1998.35'), Decimal('7.47'))
(565, datetime.date(2024, 6, 5), 'Subcription', 'Cash', 'Amazonprime', Decimal('1997.54'), Decimal('0.31'))
(496, datetime.date(2024, 5, 13), 'Health', 'Credit Card', 'Doctor Consultation', Decimal('1995.94'), Decimal('8.61'))
(930, datetime.date(2024, 10, 10), 'Vegetables', 'Credit Card', 'Weekly purchase', Decimal('1993.89'), Decimal('8.46'))
(909, datetime.date(2024, 10, 25), 'Bills', 'Credit Card', 'Gas', Decimal('1992.32'), Decimal('6.51'))
(114, datetime.date(2024, 2, 16), 'Groceries', 'Credit Card', 'Weekly purchase', Decimal('1992.11'), Decimal('0.05'))
(30, datetime.date(2024, 1, 3), 'Shopping', 'Credit Card', 'Kitchen Things', Decimal('1990.73'), Decimal('2.49'))
(1093, datetime.date(2024, 11, 24), 'Investments', 'Cash', 'Postoffice', Decimal('1989.20'), Decimal('9.77

In [21]:
#Categorie Wise Transaction
category_query = """
SELECT Categories, COUNT(*) AS Trancount 
FROM per_exp_track
GROUP BY Categories
ORDER BY Categories ASC
"""

with connection.cursor() as cursor:
    cursor.execute(category_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Bills', 146)
('Fees', 155)
('Groceries', 153)
('Health', 161)
('Investments', 152)
('Shopping', 149)
('Subcription', 140)
('Subscription', 2)
('Vegetables', 145)


In [20]:
#Categorie Wise Transaction
category_query = """
SELECT Categories, COUNT(*) AS Trancount
FROM per_exp_track
GROUP BY Categories
UNION ALL
SELECT 'Total' AS Categories, COUNT(*) AS Trancount
FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(category_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Subcription', 140)
('Vegetables', 145)
('Shopping', 149)
('Health', 161)
('Bills', 146)
('Groceries', 153)
('Fees', 155)
('Investments', 152)
('Subscription', 2)
('Total', 1203)


In [9]:
#Categorie Wise Exp Amount
category_Exp_Amt_query = """
SELECT Categories, SUM(Amount) AS Total, COUNT(*) AS Trancount
FROM per_exp_track
GROUP BY Categories
"""

with connection.cursor() as cursor:
    cursor.execute(category_Exp_Amt_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Subcription', Decimal('144708.45'), 139)
('Vegetables', Decimal('137316.74'), 145)
('Shopping', Decimal('149836.77'), 149)
('Health', Decimal('158538.72'), 161)
('Bills', Decimal('152434.81'), 146)
('Groceries', Decimal('151213.95'), 153)
('Fees', Decimal('159698.07'), 155)
('Investments', Decimal('147527.54'), 152)
('Subscription', Decimal('1278.00'), 2)


In [58]:
#Paymentmode Wise Transaction
paymode_count_query = """
SELECT Paymentmode, COUNT(*) AS Trancount 
FROM per_exp_track
GROUP BY Paymentmode
UNION ALL
SELECT 'Total' AS Paymentmode, COUNT(*) AS Trancount
FROM per_exp_track
"""

with connection.cursor() as cursor:
    cursor.execute(paymode_count_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Credit Card', 300)
('UPI', 301)
('Cash', 268)
('NEFT', 333)
('Total', 1202)


In [50]:
#Group by Month
group_month_query = """
SELECT Categories, MONTH(Date) AS Month, COUNT(*) AS Count, SUM(Amount) AS Totalexp 
FROM per_exp_track
GROUP BY Categories, MONTH(Date)
ORDER BY Categories, Month
"""

with connection.cursor() as cursor:
    cursor.execute(group_month_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Bills', 1, 13, Decimal('10875.33'))
('Bills', 2, 12, Decimal('12522.41'))
('Bills', 3, 13, Decimal('12476.03'))
('Bills', 4, 10, Decimal('11334.67'))
('Bills', 5, 10, Decimal('10381.95'))
('Bills', 6, 8, Decimal('8946.87'))
('Bills', 7, 15, Decimal('14083.25'))
('Bills', 8, 11, Decimal('10655.46'))
('Bills', 9, 16, Decimal('18541.25'))
('Bills', 10, 14, Decimal('14762.89'))
('Bills', 11, 12, Decimal('13710.75'))
('Bills', 12, 12, Decimal('14143.95'))
('Fees', 1, 10, Decimal('12001.08'))
('Fees', 2, 13, Decimal('11121.35'))
('Fees', 3, 6, Decimal('5258.50'))
('Fees', 4, 21, Decimal('24057.01'))
('Fees', 5, 14, Decimal('18753.62'))
('Fees', 6, 8, Decimal('9289.29'))
('Fees', 7, 12, Decimal('8388.88'))
('Fees', 8, 7, Decimal('6257.57'))
('Fees', 9, 10, Decimal('10371.90'))
('Fees', 10, 14, Decimal('13376.80'))
('Fees', 11, 26, Decimal('27871.88'))
('Fees', 12, 14, Decimal('12950.19'))
('Groceries', 1, 11, Decimal('12669.97'))
('Groceries', 2, 13, Decimal('10423.51'))
('Groceries', 3, 15

In [15]:
#May Month Retrieve Data
may_month_query = """
SELECT * FROM per_exp_track
WHERE Date BETWEEN '2024-05-01' AND '2024-05-31'
"""

with connection.cursor() as cursor:
    cursor.execute(may_month_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(401, datetime.date(2024, 5, 26), 'Fees', 'UPI', 'Termfee', Decimal('1000.10'), Decimal('0.66'))
(402, datetime.date(2024, 5, 4), 'Shopping', 'Cash', 'New Gadgets', Decimal('1399.10'), Decimal('5.27'))
(403, datetime.date(2024, 5, 25), 'Health', 'UPI', 'Nutrition', Decimal('895.09'), Decimal('1.92'))
(404, datetime.date(2024, 5, 18), 'Fees', 'NEFT', 'Termfee', Decimal('1644.44'), Decimal('5.34'))
(405, datetime.date(2024, 5, 30), 'Investments', 'Cash', 'Insurance', Decimal('1677.45'), Decimal('0.10'))
(406, datetime.date(2024, 5, 31), 'Shopping', 'Cash', 'New Gadgets', Decimal('1137.29'), Decimal('1.51'))
(407, datetime.date(2024, 5, 18), 'Vegetables', 'Credit Card', 'Weekly purchase', Decimal('1633.83'), Decimal('9.22'))
(408, datetime.date(2024, 5, 6), 'Bills', 'UPI', 'Wifi', Decimal('1677.11'), Decimal('2.67'))
(409, datetime.date(2024, 5, 13), 'Vegetables', 'Credit Card', 'Weekly purchase', Decimal('319.90'), Decimal('9.71'))
(410, datetime.date(2024, 5, 28), 'Bills', 'UPI', 'Gas',

In [10]:
#Insert Single Data
insert_data_query = """
INSERT INTO per_exp_track (Date, Categories, Paymentmode, Description, Amount, Cashback)
VALUES ('2024-06-12', 'Subcription', 'NEFT', 'Netflix', 639.00, 5.00) 
"""

with connection.cursor() as cursor:
    cursor.execute(insert_data_query) 
    result = cursor.fetchall()
    connection.commit()
    print(f"{cursor.rowcount} row(s) updated successfully.")


1 row(s) updated successfully.


In [11]:
#Update Data
update_data_query = """
UPDATE per_exp_track 
SET Paymentmode = 'UPI'
WHERE Description = 'Netflix' AND Date ='2024-06-12'  
"""

with connection.cursor() as cursor:
    cursor.execute(update_data_query) 
    result = cursor.fetchall()
    connection.commit()
    print(f"{cursor.rowcount} row(s) updated successfully.")

2 row(s) updated successfully.


In [12]:
#Specific Month Retrieve Data
Dec_month_query = """
SELECT * FROM per_exp_track
WHERE MONTH(Date) = 12 AND YEAR(Date) = 2024
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(Dec_month_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(1101, datetime.date(2024, 12, 8), 'Vegetables', 'NEFT', 'Weekly purchase', Decimal('1713.52'), Decimal('5.27'))
(1102, datetime.date(2024, 12, 29), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('1517.26'), Decimal('6.15'))
(1103, datetime.date(2024, 12, 17), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('24.64'), Decimal('9.77'))
(1104, datetime.date(2024, 12, 8), 'Health', 'NEFT', 'Doctor Consultation', Decimal('1572.33'), Decimal('0.77'))
(1105, datetime.date(2024, 12, 23), 'Shopping', 'Cash', 'New Furnitures', Decimal('888.74'), Decimal('5.02'))
(1106, datetime.date(2024, 12, 20), 'Subcription', 'Credit Card', 'Hotstar', Decimal('210.51'), Decimal('5.54'))
(1107, datetime.date(2024, 12, 23), 'Fees', 'Cash', 'Termfee', Decimal('449.44'), Decimal('5.24'))
(1108, datetime.date(2024, 12, 20), 'Shopping', 'Credit Card', 'New Gadgets', Decimal('1411.17'), Decimal('4.01'))
(1109, datetime.date(2024, 12, 23), 'Fees', 'NEFT', 'Coursefee', Decimal('1598.71'), Decimal('6.94'))
(1110, date

In [11]:
#Most Recent Exp for the date
Rec_date_query = """
SELECT * FROM per_exp_track
ORDER BY Date DESC
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(Rec_date_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(1172, datetime.date(2024, 12, 30), 'Health', 'UPI', 'Health Drinks', Decimal('49.59'), Decimal('3.33'))
(1170, datetime.date(2024, 12, 30), 'Health', 'NEFT', 'Gym Workout', Decimal('253.52'), Decimal('7.90'))
(1142, datetime.date(2024, 12, 30), 'Vegetables', 'Cash', 'Weekly purchase', Decimal('915.54'), Decimal('2.73'))
(1131, datetime.date(2024, 12, 29), 'Fees', 'NEFT', 'Coursefee', Decimal('1456.27'), Decimal('2.31'))
(1166, datetime.date(2024, 12, 29), 'Vegetables', 'Cash', 'Weekly purchase', Decimal('731.18'), Decimal('8.86'))
(1102, datetime.date(2024, 12, 29), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('1517.26'), Decimal('6.15'))
(1140, datetime.date(2024, 12, 29), 'Groceries', 'UPI', 'Weekly purchase', Decimal('1647.31'), Decimal('9.75'))
(1141, datetime.date(2024, 12, 29), 'Health', 'NEFT', 'Health Drinks', Decimal('1446.90'), Decimal('4.30'))
(1132, datetime.date(2024, 12, 28), 'Vegetables', 'NEFT', 'Weekly purchase', Decimal('1206.81'), Decimal('7.21'))
(1129, datetime

In [10]:
#Most Recent Exp for the Amount
Rec_exp_query = """
SELECT * FROM per_exp_track
ORDER BY Amount DESC
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(Rec_exp_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(258, datetime.date(2024, 3, 26), 'Groceries', 'NEFT', 'Weekly purchase', Decimal('1999.62'), Decimal('8.91'))
(991, datetime.date(2024, 10, 10), 'Fees', 'NEFT', 'Coursefee', Decimal('1998.35'), Decimal('7.47'))
(565, datetime.date(2024, 6, 5), 'Subcription', 'Cash', 'Amazonprime', Decimal('1997.54'), Decimal('0.31'))
(496, datetime.date(2024, 5, 13), 'Health', 'Credit Card', 'Doctor Consultation', Decimal('1995.94'), Decimal('8.61'))
(930, datetime.date(2024, 10, 10), 'Vegetables', 'Credit Card', 'Weekly purchase', Decimal('1993.89'), Decimal('8.46'))
(909, datetime.date(2024, 10, 25), 'Bills', 'Credit Card', 'Gas', Decimal('1992.32'), Decimal('6.51'))
(114, datetime.date(2024, 2, 16), 'Groceries', 'Credit Card', 'Weekly purchase', Decimal('1992.11'), Decimal('0.05'))
(30, datetime.date(2024, 1, 3), 'Shopping', 'Credit Card', 'Kitchen Things', Decimal('1990.73'), Decimal('2.49'))
(1093, datetime.date(2024, 11, 24), 'Investments', 'Cash', 'Postoffice', Decimal('1989.20'), Decimal('9.77

In [9]:
#Concat the Categories and Pamentmode
concat_query = """
SELECT CONCAT(Paymentmode, ' - ', Categories) AS Combined_Column, Amount 
FROM per_exp_track 
LIMIT 10
"""

with connection.cursor() as cursor:
    cursor.execute(concat_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

('Credit Card - Subcription', Decimal('1894.91'))
('UPI - Vegetables', Decimal('385.85'))
('UPI - Shopping', Decimal('962.18'))
('UPI - Vegetables', Decimal('1604.39'))
('Cash - Health', Decimal('1178.74'))
('NEFT - Shopping', Decimal('812.80'))
('NEFT - Bills', Decimal('1732.55'))
('Credit Card - Groceries', Decimal('818.45'))
('UPI - Vegetables', Decimal('1120.84'))
('NEFT - Subcription', Decimal('1007.73'))


In [51]:
#Shopping Expenses
shop_exp_query = """
SELECT COUNT(*) AS total_expenses, SUM(Amount) 
FROM per_exp_track 
WHERE Categories='shopping'
"""

with connection.cursor() as cursor:
    cursor.execute(shop_exp_query)
    result = cursor.fetchall()
    connection.commit()
    for row in result:
        print(row)

(149, Decimal('149836.77'))
